In [1]:
try:
    passes += 1
except NameError:
    %cd ..
    passes = 1

c:\Users\susanne\git\binarybeech


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/titanic_kaggle.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic), :]

In [4]:
from binarybeech.binarybeech import *
import binarybeech.utils as utils

In [5]:
import logging

logging.basicConfig(level=logging.INFO)

In [6]:
rf_titanic = RandomForest(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 4, "min_leaf_samples": 2, "min_split_samples": 2},
    sample_frac=0.3,
    n_attributes=5,
    metrics_type="classification",
)

In [7]:
rf_titanic.train(20)

A tree with 12 leafs was created
A tree with 11 leafs was created
A tree with 11 leafs was created
A tree with 11 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 10 leafs was created
A tree with 13 leafs was created
A tree with 9 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 5 leafs was created
A tree with 15 leafs was created
A tree with 15 leafs was created
A tree with 11 leafs was created
A tree with 8 leafs was created
A tree with 11 leafs was created
A tree with 10 leafs was created
A tree with 14 leafs was created
A tree with 12 leafs was created


In [8]:
rf_titanic.validate_oob()

{'precision': array([0.94660194, 0.46875   ]),
 'recall': array([0.74144487, 0.84507042]),
 'F-score': 0.7172857892875892,
 'accuracy': 0.7634730538922155}

In [9]:
rf_titanic.validate()

{'precision': array([0.98058252, 0.515625  ]),
 'recall': array([0.76515152, 0.94285714]),
 'F-score': 0.7631205673758865,
 'accuracy': 0.8023952095808383}

In [10]:
rf_titanic.variable_importance()

{'Sex': 1.0,
 'Pclass': 0.39145907473309616,
 'Fare': 0.26690391459074747,
 'Age': 0.24199288256227758,
 'Embarked': 0.10320284697508887,
 'PassengerId': 0.09964412811387907,
 'Parch': 0.08896797153024912,
 'SibSp': 0.06761565836298931,
 'Name': 0.0,
 'Ticket': 0.0,
 'Cabin': 0.0}

In [11]:
import binarybeech.utils as bu

bu.print_bars(rf_titanic.variable_importance(), max_width=50)

Sex       |############################### 1.0
Pclass    |############                   0.39
Fare      |########                       0.27
Age       |########                       0.24
Embarked  |###                             0.1
PassengerId|###                             0.1
Parch     |###                            0.089
SibSp     |##                             0.068
Name      |                                0.0
Ticket    |                                0.0
Cabin     |                                0.0


In [12]:
rf_titanic.validate(df_test)

{'precision': array([0.97632058, 0.48538012]),
 'recall': array([0.75280899, 0.9273743 ]),
 'F-score': 0.7436775188323559,
 'accuracy': 0.7878787878787878}

In [13]:
gbt_titanic = GradientBoostedTree(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 2, "min_leaf_samples": 10, "min_split_samples": 10},
    sample_frac=0.75,
    n_attributes=9,
)

In [14]:
gbt_titanic.X_names

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [15]:
it = gbt_titanic._initial_tree()

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


A tree with 1 leafs was created


In [16]:
utils.print_tree(it.tree)

0.6167664670658682



In [17]:
it.tree.root

In [18]:
gbt_titanic.learning_rate = 0.01
# gbt_titanic.create_trees(200)

In [19]:
gbt_titanic.predict(df_titanic.iloc[[0]])

array([0.61676647])

In [20]:
gbt_titanic.validate(df_test)

c:\Users\susanne\git\binarybeech\binarybeech\metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(m) / np.sum(m, axis=0)


{'precision': array([0., 1.]),
 'recall': array([       nan, 0.38383838]),
 'F-score': nan,
 'accuracy': 0.3838383838383838}

#k-fold cross-validation

In [21]:
c = CART(df=df_titanic, y_name="Survived", metrics_type="classification")

In [22]:
c.train(slack=1e-3)

A tree with 99 leafs was created
A tree with 92 leafs was created
A tree with 78 leafs was created
A tree with 87 leafs was created
A tree with 85 leafs was created
A tree with 92 leafs was created


INFO:binarybeech.binarybeech:beta_best: 0.005680738311679728


A tree with 99 leafs was created


In [23]:
c.tree.leaf_count()

5

In [24]:
utils.print_tree(c.tree)

Sex in ('female',)
├── False:Age<3.21
│   ├── False:0
│   └── True: 1
└── True: Pclass<2.99
    ├── False:Fare<20.82
    │   ├── False:0
    │   └── True: 1
    └── True: 1



In [25]:
c.validate(df_test)

{'precision': array([0.89253188, 0.69883041]),
 'recall': array([0.82630691, 0.80201342]),
 'F-score': 0.8025093038528897,
 'accuracy': 0.8181818181818182}